**Objective:** assess racial-ethnic mix in adolescent suicides and contrast with overall suicides.

Other research questions: evolution of the racial-ethnic distribution of adolescent suicides over time (+ relative to the proportion of the population in that age group)

In parallel: evolution of the racial-ethnic distribution of overall suicides and adult suicides (20+) over time (+ relative to the proportion of the US population -- overall or 20+)

For now, we can focus on the national level.

However, we could also assess whether differences among racial-ethnic subgroups are more pronounced in certain HHS regions and/or states.

Along these lines, health journalists at CNN, US News, and NBC were most interested in the racial-ethnic mix in the 5 states with a stat. sig. increase in the absolute number of suicides + proportion outcome as well as California (stat. sig. increase in the proportion outcome only).

**Data extraction :**

[CDC Wonder](https://wonder.cdc.gov/mcd.html), Provisional Mortality Statistics, 2018 through Last Month Request & Current Final Multiple Cause of Death Data

Groupby: Residence HHS Regions, Residence State, Year, Single Race 6, Hispanic Origin

Cause of death: Intentional self-harm

4 files:
*  All years "Overall"
*  10-19 years "10-19"
*  20-64 years "20-64"
*  20+ years "20plus"

Years after 2010

# Import package

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import datetime
from tqdm import tqdm

# Data loader

In [190]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from typing import Dict, List
color_ = [
        "#636EFA",
        "#EF553B",
        "#00CC96",
        "#AB63FA",
        "#FFA15A",
        "#19D3F3",
        "#FF6692",
        "#B6E880",
        "#FF97FF",
        "#FECB52",
    ]*10

def relabel_fig(fig):
    color = color_.copy()
    label_color = dict()

    def f(trace):
        next_color = label_color.get(trace.name, None)
        if not next_color:
            next_color = color.pop(0)
            label_color[trace.name] = next_color
            trace.line.color = next_color

        else:
            trace.line.color = next_color
            return trace.update(showlegend=False)
        return trace.update()

    label_color = dict()
    fig.for_each_trace(f)


class SuicideData:
    """Available features to plot: race, year, population, hhs, deaths, deaths_perc, etchnicity, ethno_race, age_strat."""

    def __init__(
        self,
        data_folder: str = "Data",
        drop_cols: List[str] = ["Year Code", "State Code"],
    ):
        """
        Load the files and create dataframes
        """

        self.column_cache = pd.Series(dtype="object")
        self.convert_cols = None
        self.data_folder = data_folder
        # every value detected only after 2018
        self.reject_list = [
            "American Indian or Alaska Native",
            "More than one race",
            "Native Hawaiian or Other Pacific Islander",
            "Not Stated",
        ]
        # list of unique values of a column
        self.partitions = dict()

        # will be cached
        self.data = self.load_data(drop_cols=drop_cols)
        self.merged = dict()

    def file_to_dataframe(
        self,
        data_folder: str,
        file: str,
        rename_mapper: Dict[str, str] = {
            "Single Race 6": "race",
            "Race": "race",
            "Residence HHS Region Code": "hhs",
            "Population": "population",
            "Year": "year",
            "Deaths": "deaths",
            "Hispanic Origin": "ethnicity",
        },
    ) -> pd.DataFrame:
        """
        CDC Wonder txt file into dataframe
        """
        process_line = lambda line: line.strip().replace('"', "").split("\t")

        lines = []
        with open(f"{data_folder}/{file}", "r") as f:
            for line in iter(lambda: f.readline().rstrip(), '"---"'):
                lines.append(process_line(line))
        # column is on the header, remove corner named notes
        return pd.DataFrame(lines[1:], columns=lines[0][1:]).rename(
            columns=rename_mapper
        )

    def processor(
        self,
        x: pd.DataFrame,
        force_numeric: List[str] = ["population", "Crude Rate"],
    ) -> pd.DataFrame:
        """Process dataframes."""

        if self.column_cache.empty:
            self.column_cache = x.apply(
                lambda s: pd.to_numeric(s.replace(np.nan, 0), errors="coerce")
                .notnull()
                .all()
            )

            self.convert_cols = [
                col
                for col, bool_ in self.column_cache.items()
                if (bool_ or col in force_numeric) and col != "year"
                # keep year as str to prevent unexpected ticks on plots
            ]

        x.columns = self.column_cache.index

        x.year = x.year.str.extract("(\d+)")
        # x = x.replace(["Not Applicable", "Unreliable"], np.nan)
        x = x.replace(
            {
                "Not Hispanic or Latino": "Non-Hispanic",
                "Hispanic or Latino": "Hispanic",
                "Not Applicable": np.nan,
                "Unreliable": np.nan,
                "Asian or Pacific Islander": "API",
                "Asian": "API",
                "Black or African American": "Black",
            }
        )

        x[self.convert_cols] = x[self.convert_cols].apply(
            pd.to_numeric, errors="coerce"
        )

        x = x.loc[
            ~pd.concat(
                [x.eq(forbidden).any(axis=1) for forbidden in self.reject_list], axis=1
            ).any(axis=1)
        ]

        return x.assign(ethno_race=lambda x: x.race + " " + x.ethnicity)

    def load_data(self, drop_cols: List[str] = []) -> Dict[str, pd.DataFrame]:
        available_files = os.listdir("data")
        raw_data = [file for file in available_files if "Data" in file]

        data = {
            (entry.split()[-1].split(".")[0], entry.split()[-2]): entry
            for entry in raw_data
        }

        age_strats = set(
            key[0] for key in data.keys()
        )  # {'10-19', '20plus', 'Overall'}

        dataframes = {
            age_strat: pd.concat(
                map(
                    self.processor,
                    [
                        self.file_to_dataframe(self.data_folder, file).assign(
                            age_strat=age_strat
                        )
                        for key_tuple, file in data.items()
                        if key_tuple[0] == age_strat
                    ],
                ),
                axis=0,
            ).drop(columns=drop_cols)
            for age_strat in age_strats
        }
        return dataframes

    def selection(self, subpop, df):
        """Warning: return a pointer to the slice, not a copy!"""
        return df.loc[(slice(None), slice(None), subpop)].sort_index().reset_index()

    def merge(
        self,
        x: str = "year",
        color: str = "age_strat",
        by: str = "race",
    ) -> pd.DataFrame:
        # get the list of values by
        by_list = self.partitions.get((x, by), None)
        if by_list is None:
            by_list = pd.concat(self.data.values())[by].unique()
            by_list = [k for k in by_list if k not in self.reject_list]
            by_list.sort()

            # Cache it for next time
            self.partitions[(x, by)] = by_list

            self.merged[(x, by)] = (
                pd.concat(
                    [
                        df.set_index([color, x, by])[["deaths", "population"]]
                        .groupby(level=[0, 1, 2])
                        .sum()
                        .assign(
                            suicide_per_100k=lambda x: 100000.0
                            * x.deaths
                            / x.population,
                        )
                        for key, df in self.data.items()
                    ]
                ).reset_index()
                # .rename(columns={"level_0": "age_strat"})
                .set_index([color, x, by])
            )
        return self.merged[(x, by)], by_list

    def plot(
        self,
        x: str = "year",
        y: str = "deaths",
        color: str = "age_strat",
        by: str = "race",
        scatter: bool = False,
        rows: int = 2,
    ) -> None:

        _, by_list = self.merge(x=x, color=color, by=by)

        # adjust the number of cols for the plot
        cols = len(by_list) // rows + 1 if len(by_list) % rows else len(by_list) // rows

        fig = make_subplots(
            rows=rows, cols=cols, subplot_titles=tuple(map(lambda x: str(x), by_list))
        )

        for i, subpop in enumerate(by_list):
            sub_df = self.selection(subpop, self.merged[(x, by)])
            plot_method = px.line
            sub_fig = plot_method(
                sub_df.sort_values(by=[x]),
                x=x,
                y=y,
                color=color,
                title="test",  # str(subpop),
            )

            # merge px express on subplot
            for d in sub_fig.data:
                fig.add_trace(
                    (go.Scatter(x=d["x"], y=d["y"], name=d["name"])),
                    row=(1 + i // cols),
                    col=(1 + i % cols),
                )
        if scatter:
            fig.update_traces(line=dict(width=0))
        relabel_fig(fig)
        fig.update_layout(
            title_text="{}Evolution of {} by {}".format(
                "Temporal " if x == "year" else "", y, by
            ),
            xaxis_title=x,
            yaxis_title=y,
            legend_title=by,
            height=330 * rows,
            width=400 * cols,
        )
        fig.update_xaxes(tickangle=-45)
        fig.write_image("outputs/{}_{}_by_{}_color_{}.png".format(y, x, by, color))
        fig.show()

In [158]:
sd = SuicideData()

In [159]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

data, _ = sd.merge(x="year", color="age_strat", by="ethnicity")

data = (
    data.assign(
        pop_share=lambda x: 100.0
        * x.population
        / x.groupby(level=[0, 1]).sum().population
    )
    .reset_index()
    .set_index(["ethnicity", "year", "age_strat"])
)

In [171]:
df#.suicide_per_100k[#df.ethnicity=="Hispanic"]

,ethnicity,year,deaths,population,suicide_per_100k,pop_share
0,Hispanic,2010,1980,24372202.0,8.124009,13.731153
12,Non-Hispanic,2010,27696,153123467.0,18.087365,86.268847
1,Hispanic,2011,2048,25297378.0,8.095701,14.092682
13,Non-Hispanic,2011,28305,154209822.0,18.354862,85.907318
2,Hispanic,2012,2141,26027275.0,8.225986,14.412147
14,Non-Hispanic,2012,28887,154565347.0,18.689183,85.587853
3,Hispanic,2013,2144,26256013.0,8.165749,14.481460
15,Non-Hispanic,2013,28801,155051760.0,18.575087,85.518540
4,Hispanic,2014,2453,28041195.0,8.747844,15.349283
16,Non-Hispanic,2014,29562,154646139.0,19.115899,84.650717


In [174]:
df = sd.selection("20-64", data).sort_values("year")

fig = make_subplots(specs=[[{"secondary_y": True}]])

for request in ["Hispanic", "Non-Hispanic"]:
    sub_df = df[df.ethnicity==request]
    # Add traces
    fig.add_trace(
        go.Scatter(x=sub_df.year, y=sub_df.suicide_per_100k, name="yaxis data"),
        secondary_y=False,
    )




    fig.add_trace(
        go.Scatter(x=sub_df.year, y=sub_df.pop_share, name="yaxis2 data",line=dict(dash='dot'),),
        secondary_y=True,
    )

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

In [118]:
data

deaths   population  suicide_per_100k  pop_share
age_strat year ethnicity                                                      
20-64     2010  Hispanic        1980   24372202.0          8.124009  13.731153
                Non-Hispanic   27696  153123467.0         18.087365  86.268847
          2011  Hispanic        2048   25297378.0          8.095701  14.092682
                Non-Hispanic   28305  154209822.0         18.354862  85.907318
          2012  Hispanic        2141   26027275.0          8.225986  14.412147
...                              ...          ...               ...        ...
Overall   2019  Non-Hispanic   41794  255723689.0         16.343421  82.973595
          2020  Hispanic        4261   53426611.0          7.975426  17.285455
                Non-Hispanic   39949  255657595.0         15.625978  82.714545
          2021  Hispanic        3418   53107646.0          6.435985  17.235209
                Non-Hispanic   31101  255026977.0         12.195180  82.764791

[96 rows x 4 columns]

In [199]:
sd = SuicideData()
queue = [
    {
        "x": "year",
        "y": "suicide_per_100k",
        "by": "ethno_race",
        "color": "age_strat",
        "scatter": False,
        "rows": 3,
    },
    {
        "x": "year",
        "y": "suicide_per_100k",
        "by": "race",
        "color": "age_strat",
        "scatter": False,
        "rows": 3,
    },
    {
        "x": "year",
        "y": "population",
        "by": "race",
        "color": "age_strat",
        "scatter": False,
        "rows": 1,
    },
    {
        "x": "year",
        "y": "population",
        "color": "race",
        "by": "age_strat",
        "scatter": False,
        "rows": 1,
    },
    {
        "x": "age_strat",
        "y": "suicide_per_100k",
        "color": "race",
        "by": "year",
        "scatter": True,
        "rows": 4,
    },
    {
        "x": "age_strat",
        "y": "suicide_per_100k",
        "color": "year",
        "by": "race",
        "scatter": True,
        "rows": 1,
    },
    {
        "x": "ethno_race",
        "y": "suicide_per_100k",
        "color": "year",
        "by": "age_strat",
        "scatter": True,
        "rows": 2,
    },
]
[sd.plot(**el) for el in queue]

[None, None, None, None, None, None, None]

In [66]:
sd.merged[('year','ethno_race')]

deaths   population  suicide_per_100k
age_strat year ethno_race                                                
20-64     2010 API  Hispanic            10      31688.0         31.557687
               API  Non-Hispanic       733    9055143.0          8.094847
               Black  Non-Hispanic    1728   22779776.0          7.585676
               White  Hispanic        1970   24340514.0          8.093502
               White  Non-Hispanic   25235  121288548.0         20.805757
...                                    ...          ...               ...
Overall   2021 API  Non-Hispanic       919   17615053.0          5.217129
               Black  Hispanic          10     624321.0          1.601740
               Black  Non-Hispanic    2728   40638534.0          6.712841
               White  Hispanic        3408   52483325.0          6.493491
               White  Non-Hispanic   27454  196773390.0         13.952090

[222 rows x 3 columns]

In [686]:
x = "year"
y = "suicide_perc"
by = "race"
color = "age_strat"
pd.concat(
    [
        df.assign(ethno_race=lambda x: x.race  + x.ethnicity)
        .set_index([color, x, by])[["deaths", "Population"]]
        #.groupby(level=[0, 1, 2])
        #.sum()
        #.assign(
        #    suicide_perc=lambda x: 100.0 * x.deaths / x.Population,
        #)
        # .assign(ethno_race=lambda x: x.race + " " + x.ethnicity)
        for key, df in sd.data.items()
    ]
)#.reset_index()
# .rename(columns={"level_0": "age_strat"})
# .set_index([color, x, by])

,Residence HHS Region,hhs,Residence State,Residence State Code,year,Year Code,race,Single Race 6 Code,ethnicity,Hispanic Origin Code,deaths,Population,Crude Rate,age_strat,ethno_race
0,"HHS Region #1 CT, ME, MA, NH, RI, VT",HHS1,Connecticut,9,2018,2018,Black or African American,2054-5,Non-Hispanic,2186-2,23,269173.0,8.5,20plus,Black or African American Non-Hispanic
1,"HHS Region #1 CT, ME, MA, NH, RI, VT",HHS1,Connecticut,9,2018,2018,White,2106-3,Hispanic,2135-2,32,321427.0,10.0,20plus,White Hispanic
2,"HHS Region #1 CT, ME, MA, NH, RI, VT",HHS1,Connecticut,9,2018,2018,White,2106-3,Non-Hispanic,2186-2,344,1912405.0,18.0,20plus,White Non-Hispanic
3,"HHS Region #1 CT, ME, MA, NH, RI, VT",HHS1,Connecticut,9,2019,2019,Black or African American,2054-5,Non-Hispanic,2186-2,33,274133.0,12.0,20plus,Black or African American Non-Hispanic
4,"HHS Region #1 CT, ME, MA, NH, RI, VT",HHS1,Connecticut,9,2019,2019,White,2106-3,Hispanic,2135-2,31,327771.0,9.5,20plus,White Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,"HHS Region #10 AK, ID, OR, WA",HHS10,Washington,53,2018,2018,White,2106-3,Non-Hispanic,2186-2,49,519992.0,9.4,10-19,White Non-Hispanic
266,"HHS Region #10 AK, ID, OR, WA",HHS10,Washington,53,2019,2019,White,2106-3,Non-Hispanic,2186-2,42,516509.0,8.1,10-19,White Non-Hispanic
267,"HHS Region #10 AK, ID, OR, WA",HHS10,Washington,53,2020,2020,White,2106-3,Hispanic,2135-2,14,166154.0,NaN,10-19,White Hispanic
268,"HHS Region #10 AK, ID, OR, WA",HHS10,Washington,53,2020,2020,White,2106-3,Non-Hispanic,2186-2,37,512801.0,7.2,10-19,White Non-Hispanic


In [682]:
for df in sd.data.values():
    print(df.columns)

Index(['Residence HHS Region', 'hhs', 'Residence State',
       'Residence State Code', 'year', 'Year Code', 'race',
       'Single Race 6 Code', 'ethnicity', 'Hispanic Origin Code', 'deaths',
       'Population', 'Crude Rate', 'age_strat'],
      dtype='object')
Index(['Residence HHS Region', 'hhs', 'Residence State',
       'Residence State Code', 'year', 'Year Code', 'race',
       'Single Race 6 Code', 'ethnicity', 'Hispanic Origin Code', 'deaths',
       'Population', 'Crude Rate', 'age_strat'],
      dtype='object')
Index(['Residence HHS Region', 'hhs', 'Residence State',
       'Residence State Code', 'year', 'Year Code', 'race',
       'Single Race 6 Code', 'ethnicity', 'Hispanic Origin Code', 'deaths',
       'Population', 'Crude Rate', 'age_strat'],
      dtype='object')


In [563]:
pd.concat(
    {
        key: df.set_index(["Year", "Single Race 6"])[["Deaths", "Population"]]
        .groupby(level=[0, 1])
        .sum()
        .assign(suicide_rate=lambda x: x.Deaths / x.Population)
        for key, df in sd.data.items()
    }
)

Deaths   Population  \
       Year Single Race 6                                           
20plus 2015 American Indian or Alaska Native     362    1188425.0   
            Asian or Pacific Islander           1101   13557779.0   
            Black or African American           2135   29097926.0   
            White                              37673  189118506.0   
       2016 American Indian or Alaska Native     419    1297492.0   
...                                              ...          ...   
10-19  2020 White                               1964   27119988.0   
       2021 American Indian or Alaska Native      41      86362.0   
            Asian                                 24     792912.0   
            Black or African American            141    2702938.0   
            White                               1455   26043383.0   

                                              suicide_rate  
       Year Single Race 6                                   
20plus 2015 American Indian or Alaska Native      0.000305  
            Asian or Pacific Islander             0.000081  
            Black or African American             0.000073  
            White                                 0.000199  
       2016 American Indian or Alaska Native      0.000323  
...                                                    ...  
10-19  2020 White                                 0.000072  
       2021 American Indian or Alaska Native      0.000475  
            Asian                                 0.000030  
            Black or African American             0.000052  
            White                                 0.000056  

[103 rows x 3 columns]

In [380]:
sd.merged["hhs"]

Deaths  Population  suicide_rate
       Year hhs                                    
20plus 2015 HHS1     1606  10803223.0      0.000149
            HHS10    2243   9885835.0      0.000227
            HHS2     2331  20973300.0      0.000111
            HHS3     3834  22690519.0      0.000169
            HHS4     9176  47507129.0      0.000193
...                   ...         ...           ...
10-19  2021 HHS5      264   5201953.0      0.000051
            HHS6      328   5202417.0      0.000063
            HHS7       91   1345176.0      0.000068
            HHS8      135   1175738.0      0.000115
            HHS9      188   5066419.0      0.000037

[210 rows x 3 columns]

In [383]:
sd.partitions["hhs"]

array(['HHS1', 'HHS2', 'HHS3', 'HHS4', 'HHS5', 'HHS6', 'HHS7', 'HHS8',
       'HHS9', 'HHS10'], dtype=object)

In [3]:
data_folder = "Data"  # where the txt files are
sd = SuicideData(data_folder)

In [4]:
sd.plot_race_data_deaths()

In [5]:
sd.plot_race_data_rate()

In [6]:
sd.plot_HHS_data_rate()